In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Parameters

modes = ['vae','classall','classred','syllall','syllred','phonall','phonred','siamese'] # Triplet!!
#clfs = ['slp','mlp','logr','knn','rf','xgboost']#
clfs = ['knn']

list_test_participants_avp = [8,10,18,23]
list_test_participants_lvt = [0,6,7,13]

# Calculate utterance-wise weights

num_test_utterances = []
for part in list_test_participants_avp:
    if part<=9:
        test_dataset = np.load('../../data/interim/AVP/Dataset_Test_0' + str(part) + '.npy')
    else:
        test_dataset = np.load('../../data/interim/AVP/Dataset_Test_' + str(part) + '.npy')
    num_test_utterances.append(test_dataset.shape[0])
for part in list_test_participants_lvt:
    if part<=9:
        test_dataset = np.load('../../data/interim/LVT/Dataset_Test_0' + str(part) + '.npy')
    else:
        test_dataset = np.load('../../data/interim/LVT/Dataset_Test_' + str(part) + '.npy')
    num_test_utterances.append(test_dataset.shape[0])
utterance_wise_weights = num_test_utterances/np.sum(np.array(num_test_utterances))

In [4]:
# Results

for b in range(len(modes)):

    for c in range(len(clfs)):

        mode = modes[b]
        clf = clfs[c]

        accuracies_raw = np.load('../../results/' + mode + '/accuracies.npy')
        accuracies_mean = np.mean(np.mean(np.mean(accuracies_raw,axis=-1),axis=-1),axis=-1)
        accuracies_std = np.std(np.mean(np.mean(accuracies_raw,axis=-1),axis=-1),axis=-1)
        
        print([mode,clf])
        print([accuracies_mean[0],accuracies_std[0]])

['vae', 'knn']
[0.7730113438870969, 0.023887072282731424]
['classall', 'knn']
[0.7858345558020263, 0.024357586246602406]
['classred', 'knn']
[0.741238013089875, 0.039757954343512675]
['syllall', 'knn']
[0.8742735089334246, 0.022817869833435873]
['syllred', 'knn']
[0.854391218665137, 0.028331635789420072]
['phonall', 'knn']
[0.8593127258705074, 0.02590090231384071]
['phonred', 'knn']
[0.8654163737664936, 0.026183922613394025]
['siamese', 'knn']
[0.8492785796940214, 0.02143377960096325]


In [5]:
# Results Utterance-Wise

for b in range(len(modes)):

    for c in range(len(clfs)):

        mode = modes[b]
        clf = clfs[c]

        accuracies_raw = np.load('../../results/' + mode + '/accuracies.npy')

        for i in range(accuracies_raw.shape[0]):
            for j in range(accuracies_raw.shape[1]):
                for k in range(accuracies_raw.shape[2]):
                    accuracies_raw[i,j,k] *= utterance_wise_weights*8

        accuracies_mean = np.mean(np.mean(np.mean(accuracies_raw,axis=-1),axis=-1),axis=-1)
        accuracies_std = np.std(np.mean(np.mean(accuracies_raw,axis=-1),axis=-1),axis=-1)
        
        print([mode,clf])
        print([accuracies_mean[0],accuracies_std[0]])

['vae', 'knn']
[0.7826677316293928, 0.02838431601115306]
['classall', 'knn']
[0.7687699680511182, 0.02727434048270364]
['classred', 'knn']
[0.7106230031948881, 0.03483389746174469]
['syllall', 'knn']
[0.846884984025559, 0.02841037584123069]
['syllred', 'knn']
[0.829552715654952, 0.03798794594061579]
['phonall', 'knn']
[0.8304313099041533, 0.031734604919298866]
['phonred', 'knn']
[0.8359424920127794, 0.03104389180771539]
['siamese', 'knn']
[0.819408945686901, 0.01384556500486244]
